In [162]:
import pandas as pd
import requests

In [163]:
# From tutorial: http://www.bd-econ.com/imfapi1.html

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.JP.PMP_IX.?startPeriod=1999' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

baseyr = data['@BASE_YEAR']  # Save the base year

# Create pandas dataframe from the observations
data_list = [[obs.get('@TIME_PERIOD'), obs.get('@OBS_VALUE')]
             for obs in data['Obs']]

df = pd.DataFrame(data_list, columns=['date', 'value'])
     
df = df.set_index(pd.to_datetime(df['date']))['value'].astype('float')
df

date
1999-01-01    73.830237
1999-02-01    74.676699
1999-03-01    75.335058
1999-04-01    75.993416
1999-05-01    77.780390
                ...    
2021-06-01    85.586645
2021-07-01    86.150952
2021-08-01    86.621209
2021-09-01    86.715260
2021-10-01    86.527157
Name: value, Length: 267, dtype: float64

In [164]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'Dataflow'  # Method with series information
search_term = 'Trade'  # Term to find in series names
series_list = requests.get(f'{url}{key}').json()\
            ['Structure']['Dataflows']['Dataflow']

# Use dict keys to navigate through results:
for series in series_list:
    if search_term in series['Name']['#text']:
        print(f"{series['Name']['#text']}: {series['KeyFamilyRef']['KeyFamilyID']}")

Direction of Trade Statistics (DOTS), 2020 Q1: DOT_2020Q1
Direction of Trade Statistics (DOTS), 2018 Q3: DOT_2018Q3
Direction of Trade Statistics (DOTS), 2018 Q2: DOT_2018Q2
Direction of Trade Statistics (DOTS), 2019 Q1: DOT_2019Q1
Direction of Trade Statistics (DOTS), 2017 Q2: DOT_2017Q2
Direction of Trade Statistics (DOTS), 2018 Q1: DOT_2018Q1
Direction of Trade Statistics (DOTS), 2019 Q4: DOT_2019Q4
Direction of Trade Statistics (DOTS), 2017 Q4: DOT_2017Q4
Direction of Trade Statistics (DOTS), 2019 Q2: DOT_2019Q2
Direction of Trade Statistics (DOTS), 2020 Q2: DOT_2020Q2
Direction of Trade Statistics (DOTS), 2018 Q4: DOT_2018Q4
Direction of Trade Statistics (DOTS), 2019 Q3: DOT_2019Q3
Direction of Trade Statistics (DOTS), 2017 Q3: DOT_2017Q3
Direction of Trade Statistics (DOTS), 2017 Q1: DOT_2017Q1
Direction of Trade Statistics (DOTS), 2020 Q4: DOT_2020Q4
Direction of Trade Statistics (DOTS), 2020 Q3: DOT_2020Q3
Direction of Trade Statistics (DOTS), 2021 Q1: DOT_2021Q1
Direction of T

In [165]:
key = 'DataStructure/DOT'  # Method / series
dimension_list = requests.get(f'{url}{key}').json()['Structure']['KeyFamilies']['KeyFamily']['Components']['Dimension']
for n, dimension in enumerate(dimension_list):
    print(f'Dimension {n+1}: {dimension["@codelist"]}')

Dimension 1: CL_FREQ
Dimension 2: CL_AREA_DOT
Dimension 3: CL_INDICATOR_DOT
Dimension 4: CL_COUNTERPART_AREA_DOT


In [166]:
# Example: codes for third dimension, which is 2 in python
key = f"CodeList/{dimension_list[2]['@codelist']}"
code_list = requests.get(f'{url}{key}').json()\
	    ['Structure']['CodeLists']['CodeList']['Code']
for code in code_list:
    print(f"{code['Description']['#text']}: {code['@value']}")

Goods, Value of Exports, Free on board (FOB), US Dollars: TXG_FOB_USD
Goods, Value of Imports, Cost, Insurance, Freight (CIF), US Dollars: TMG_CIF_USD
Goods, Value of Imports, Free on board (FOB), US Dollars: TMG_FOB_USD
Goods, Value of Trade Balance, US Dollars: TBG_USD


## CSV import

Bulk download from: https://data.imf.org/?sk=388dfa60-1d26-4ade-b505-a05a558d9a42&sid=1479329334655

In [167]:
all_countries = ['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
                 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
                 'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore',
                 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland',
                 'New Zealand', 'Austria', 'Portugal', 
                 'China', 'India', 'Korea', 'Brazil',
                 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia',
                 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye',
                 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru',
                 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']

# All countries apart from 'Eurozone' and 'Taiwan'

In [168]:
def filter_countries(df):
    df.index = df['Country Name']
    df.index.name = None
    df.rename(index={
            'Korea, Rep. of': 'Korea',
            'Türkiye, Rep of': 'Turkiye',
            'Poland, Rep. of': 'Poland',
            'Czech Rep.': 'Czechia',
            'Netherlands, The': 'Netherlands',
            'China, P.R.: Hong Kong': 'Hong Kong',
            'Russian Federation': 'Russia',
            'China, P.R.: Mainland': 'China',
            'Egypt, Arab Rep. of': 'Egypt'
        }, inplace=True)
    df = df.loc[all_countries]
    return df

In [178]:
def check_data_coverage(df):
    available_countries = df.columns[~df.isna().all()]
    print('Available countries:')
    print(available_countries)
    print()

    print('No data:')
    print('Eurozone')
    print('Taiwan')
    for country in all_countries:
        if not country in df.columns:
            print(country)
    print()

    print('Missing first:')
    for country in df.columns:
        first_valid = df[country].first_valid_index()
        if first_valid != None and (not str(first_valid).startswith('1999')):
            print(country, first_valid)
    print()

    print('Missing last:')
    for country in df.columns:
        last_valid = df[country].last_valid_index()
        if (not str(last_valid).startswith('2023')) and (not str(last_valid).startswith('2022')):
            print(country, last_valid)
            
    print()
    print('Done')

In [179]:
def read_indicator(filename, indicator, filter_period, short_name):
    df = pd.read_csv(filename)
    filtered_data = filter_countries(df)
    filtered_data = filtered_data[filtered_data['Indicator Name'] == indicator]

    filtered_data = filtered_data.loc[~filtered_data[filter_period].isna()]
    filtered_data = filtered_data.dropna(thresh=10, axis=1)
    filtered_data = filtered_data.iloc[:,5:]
    filtered_data = filtered_data.T
    filtered_data.index = pd.to_datetime(filtered_data.index)
    filtered_data = filtered_data['1999':]

    filtered_data.to_csv('IMF_'+short_name)
    check_data_coverage(filtered_data)

    return filtered_data

In [180]:
filename = 'RawData/IFS_09-23-2023 15-20-12-31_timeSeries.csv'
indicator = 'Unemployment, Persons, Percentage change, corresponding period previous year, Percent'

read_indicator(filename, indicator, '2019Q1', 'Unemployment QoQ')

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa',
       'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'Qatar', 'Turkiye',
       'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary',
       'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Eurozone
Taiwan
China
United Arab Emirates
Kuwait

Missing first:
India 2018-07-01 00:00:00
Brazil 2013-04-01 00:00:00
Saudi Arabia 2008-01-01 00:00:00
South Africa 2001-01-01 00:00:00
Thailand 2002-01-01 00:00:00
Indonesia 2009-01-01 00:00:00
Qatar 2014-01-01 00:00:00
Peru 2002-04-01 00:00:00
Egypt 2004-01-01 00:00:00
Colombia 2002-07-01 00:00:00
Argentina 2004-07-01 

C:\Users\lukas\AppData\Local\Temp\ipykernel_11564\2081141812.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_data.index = pd.to_datetime(filtered_data.index)


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,-6.780711,24.835742,-1.221141,-6.373300,0.281506,NaN,NaN,-8.644816,-19.047619,0.427350,...,20.564741,57.279713,-2.749016,NaN,-20.452567,42.872688,NaN,NaN,NaN,17.647059
1999-04-01,-2.649080,15.916955,-2.246321,-2.816052,NaN,-14.261137,-9.065913,-10.133470,-17.440913,4.292929,...,32.334508,79.485477,10.965517,NaN,-21.240602,43.982495,NaN,NaN,NaN,8.641975
1999-07-01,-5.183741,10.904872,-3.043236,-5.678410,NaN,NaN,NaN,-10.941669,-15.490376,9.582310,...,38.798799,70.866263,9.134615,NaN,-23.361823,33.618234,NaN,NaN,NaN,1.162791
1999-10-01,-6.485565,4.683841,-3.600236,-12.886918,NaN,NaN,NaN,-8.812328,-13.573407,15.831135,...,40.899601,26.223876,11.776187,NaN,-15.524625,24.098505,NaN,NaN,NaN,-8.247423
2000-01-01,-3.200154,3.035547,-6.026656,-12.466117,-14.107586,NaN,NaN,-4.477807,-11.140820,-9.574468,...,31.972265,1.501597,1.148730,NaN,-6.455142,12.871287,NaN,NaN,NaN,-14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01,-35.479732,-5.872193,NaN,-30.125416,-7.945058,-20.581395,-21.695238,-30.804996,-23.170732,-26.989619,...,-22.028142,NaN,-13.193202,NaN,-15.689382,-27.323420,NaN,NaN,-30.507716,-24.603175
2022-04-01,-37.917519,-9.744409,NaN,NaN,-4.922583,-18.292683,-17.404997,-24.877352,-21.360000,-15.046296,...,-25.178670,NaN,-19.208011,NaN,-14.864865,-20.631579,NaN,NaN,NaN,-20.535714
2022-07-01,-28.529207,-6.250000,NaN,NaN,-6.397120,-16.184211,-5.238971,-21.781371,-5.938038,-0.669643,...,-4.921136,NaN,-9.647696,NaN,-7.732865,-14.645309,NaN,NaN,NaN,-13.861386
2022-10-01,NaN,-6.529851,NaN,NaN,-2.656023,-6.505295,-3.000492,-22.923388,-3.747715,6.896552,...,0.268097,NaN,-9.606044,NaN,0.000000,3.098592,NaN,NaN,NaN,-13.402062


In [181]:
filename = 'RawData/IFS_09-23-2023 15-20-12-31_timeSeries.csv'
indicator = 'Unemployment, Persons, Percentage change, previous period, Percent'

read_indicator(filename, indicator, '2019Q1', 'Unemployment YoY')

Available countries:
Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark',
       'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland',
       'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria',
       'Portugal', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa',
       'Mexico', 'Thailand', 'Malaysia', 'Qatar', 'Turkiye', 'Philippines',
       'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt',
       'Colombia', 'Argentina', 'Russia'],
      dtype='object')

No data:
Eurozone
Taiwan
China
Indonesia
United Arab Emirates
Kuwait

Missing first:
France 2003-04-01 00:00:00
Switzerland 2010-04-01 00:00:00
Germany 2005-04-01 00:00:00
India 2017-10-01 00:00:00
Brazil 2012-07-01 00:00:00
Saudi Arabia 2015-04-01 00:00:00
South Africa 2008-04-01 00:00:00
Mexico 2000-07-01 00:00:00
Thailand 2001-04-01 00:00:00
Qatar 2013-04-01 00:00:00
Turkiye 2000-01-01 0

C:\Users\lukas\AppData\Local\Temp\ipykernel_11564\2081141812.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_data.index = pd.to_datetime(filtered_data.index)


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,12.743907,11.241218,1.849302,11.369280,NaN,NaN,NaN,7.989420,3.601108,24.010554,...,23.173278,12.804180,7.612232,NaN,-2.141328,15.479332,NaN,NaN,NaN,3.092784
1999-04-01,-9.435395,5.789474,-2.491791,-3.488689,NaN,NaN,NaN,-10.343371,-9.714795,-12.127660,...,4.114022,31.470084,-2.720677,NaN,-8.315098,0.228484,NaN,NaN,NaN,-12.000000
1999-07-01,1.943971,-4.875622,3.506339,-5.495995,NaN,NaN,NaN,-2.017166,-8.983218,7.990315,...,2.604706,6.772517,-1.243008,NaN,-3.699284,6.914894,NaN,NaN,NaN,-1.136364
1999-10-01,-10.160871,-6.485356,-6.220096,-14.239059,NaN,NaN,NaN,-3.878062,1.518438,-1.569507,...,7.082071,-20.286864,8.118313,NaN,-2.230483,0.284293,NaN,NaN,NaN,2.298851
2000-01-01,16.704901,9.489676,-0.714286,11.907250,NaN,NaN,NaN,13.122596,6.517094,-3.189066,...,15.369073,-9.289710,-2.619325,NaN,8.365019,5.031892,NaN,NaN,NaN,-3.370787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01,5.192639,1.679104,NaN,6.689687,-0.783745,3.328290,1.106739,0.128344,-2.102377,3.940887,...,7.707775,NaN,4.749056,NaN,-3.096539,10.140845,NaN,NaN,-0.563559,-2.061856
2022-04-01,-13.971800,3.669725,NaN,NaN,-4.783499,-11.713031,-4.305522,-12.771113,-8.216620,-13.033175,...,-15.308027,NaN,-8.552293,NaN,-5.263158,-3.580563,NaN,NaN,NaN,-6.315789
2022-07-01,3.631213,-4.424779,NaN,NaN,3.856199,5.638474,4.829690,-8.275831,11.190234,21.253406,...,10.727406,NaN,-6.084507,NaN,4.166667,-1.061008,NaN,NaN,NaN,-2.247191
2022-10-01,NaN,-7.222222,NaN,NaN,-0.784024,-2.982732,-4.364694,-3.789723,-3.659652,-2.471910,...,-0.729927,NaN,0.479904,NaN,4.571429,-1.876676,NaN,NaN,NaN,-3.448276
